## Code based on PyTorch Tensors example ##
### https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html ###

In [1]:
#torch.nn provides all the building blocks for neural networks.

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else 'cpu'
print(f"Using {device} device")

Using mps device


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits       

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
#To use the model, we pass it the input data. Do not call model.forward() directly!
#Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, and dim=1 corresponding to the individual values of each output. We get the prediction probabilities by passing it through an instance of the nn.Softmax module.
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred} and predicted probability: {pred_probab}")

Predicted class: tensor([3], device='mps:0') and predicted probability: tensor([[0.0960, 0.0985, 0.0932, 0.1142, 0.0906, 0.1050, 0.1123, 0.1050, 0.0923,
         0.0928]], device='mps:0', grad_fn=<SoftmaxBackward0>)


In [9]:
#sample minibatch of 3 images of size 28x28 
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [10]:
#We initialize the nn.Flatten layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values ( the minibatch dimension (at dim=0) is maintained).
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [12]:
#The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [13]:
#Non-linear activations are what create the complex mappings between the model’s inputs and outputs. They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.0418, -0.0082,  0.0650, -0.5697, -0.1265, -0.2065,  0.0505,  0.5342,
          0.3158,  0.0881,  0.0737, -0.1546, -0.0927,  0.3216, -0.6124,  0.3702,
         -0.0636, -0.5036, -0.1843,  0.3864],
        [-0.0387, -0.1153,  0.3258, -0.1798, -0.1718, -0.1410,  0.1940,  0.2277,
          0.3622, -0.3709, -0.3134,  0.2024,  0.0814,  0.0763, -0.6555,  0.3418,
          0.0539, -0.6445, -0.3449,  0.0978],
        [ 0.1053, -0.2305,  0.1497, -0.0612, -0.2262, -0.0476,  0.0914,  0.4699,
         -0.0464,  0.1042, -0.1497, -0.1919,  0.5446,  0.2746, -0.5642,  0.6542,
         -0.2420, -0.5799, -0.1397,  0.4631]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0418, 0.0000, 0.0650, 0.0000, 0.0000, 0.0000, 0.0505, 0.5342, 0.3158,
         0.0881, 0.0737, 0.0000, 0.0000, 0.3216, 0.0000, 0.3702, 0.0000, 0.0000,
         0.0000, 0.3864],
        [0.0000, 0.0000, 0.3258, 0.0000, 0.0000, 0.0000, 0.1940, 0.2277, 0.3622,
         0.0000, 0.0000, 0.2024, 0.0814, 0.0763, 0.00

In [15]:
#nn.Sequential is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like seq_modules.
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [16]:
#The last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the nn.Softmax module. The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class. dim parameter indicates the dimension along which the values must sum to 1.
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [19]:
#Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training. Subclassing nn.Module automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model’s parameters() or named_parameters() methods.
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0124,  0.0263, -0.0233,  ..., -0.0331,  0.0224,  0.0349],
        [ 0.0159,  0.0012, -0.0345,  ..., -0.0092,  0.0276, -0.0180]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0057, -0.0306], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0155,  0.0014,  0.0119,  ...,  0.0397, -0.0111,  0.0367],
        [-0.0376,  0.0091, -0.0087,  ...,  0.0230,  0.0430, -0.0085]],
       device='mps:0', grad_fn=<Slice